<a href="https://colab.research.google.com/github/Ampiso4/Torrent-To-Google-Drive-Multi-files/blob/main/CustomTorrentToDrive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Google Colab script: download a torrent from a MAGNET link and save into
# Google Drive under a folder named "torrents"
#
# IMPORTANT: Only download content you have the right to download. Do not
# use this script to infringe copyright. Colab sessions can disconnect or
# time out — for long-running torrents a VPS or dedicated seedbox is more
# reliable.

# ---------- Cell 1: Mount Google Drive ----------
from google.colab import drive
import os

drive.mount('/content/drive')

# Path on the mounted Drive where torrents will be saved
DRIVE_TORRENTS_PATH = '/content/drive/MyDrive/torrents'
os.makedirs(DRIVE_TORRENTS_PATH, exist_ok=True)
print(f"Torrents folder (on Drive) ready at: {DRIVE_TORRENTS_PATH}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Torrents folder (on Drive) ready at: /content/drive/MyDrive/torrents


In [3]:


# ---------- Cell 2: Install aria2 (lightweight torrent client) ----------
# aria2 supports magnet links and is simple to call from the command line.
# Install via apt. This may take 10-20s.

!apt-get update -qq
!apt-get install -y -qq aria2

print('aria2 installed')


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
aria2 installed


In [ ]:


# ---------- Cell 3: Download a magnet link (blocking) ----------
# Replace the MAGNET_LINK below with the magnet URI you want to download.
# This cell will block (show progress) until aria2 finishes or the Colab
# session is disconnected. If you prefer a non-blocking background start,
# see the alternate example below.

MAGNET_LINK = "magnet:?xt=urn:btih:c5fa1e4f53b5883656177bc294a662ea5822fa6d&dn=Miasma%20Chronicles%20%5BDODI%20Repack%5D&tr=udp%3A%2F%2F9.rarbg.to%3A2870%2Fannounce&tr=udp%3A%2F%2Feddie4.nl%3A6969%2Fannounce&tr=udp%3A%2F%2Fthetracker.org%3A80%2Fannounce&tr=udp%3A%2F%2Ftracker.coppersurfer.tk%3A6969&tr=udp%3A%2F%2Ftracker.leechers-paradise.org%3A6969&tr=udp%3A%2F%2Ftracker.opentrackr.org%3A1337&tr=udp%3A%2F%2Ftracker.zer0day.to%3A1337&tr=udp%3A%2F%2F9.rarbg.com%3A2710%2Fannounce&tr=udp%3A%2F%2Fexplodie.org%3A6969%2Fannounce&tr=udp%3A%2F%2Ftracker1.wasabii.com.tw%3A6969%2Fannounce&tr=udp%3A%2F%2Ftracker.torrent.eu.org%3A451%2Fannounce&tr=udp%3A%2F%2Ftracker.cypherpunks.ru%3A6969%2Fannounce&tr=udp%3A%2F%2Fp4p.arenabg.com%3A1337%2Fannounce"

if MAGNET_LINK == "MAGNET_LINK_HERE":
    print('Please edit MAGNET_LINK and re-run this cell.')
else:
    # aria2c parameters you can tweak:
    #  --dir=DIR           -> download destination
    #  --continue=true     -> resume partial downloads
    #  --max-connection-per-server / --split  -> control connections
    #  --seed-time=0       -> stop seeding after download (0 = disable seeding time limit)
    #  --summary-interval  -> how often aria2 prints a summary (seconds)
    cmd = [
        'aria2c',
        '--dir=' + DRIVE_TORRENTS_PATH,
        '--continue=true',
        '--max-connection-per-server=4',
        '--split=4',
        '--min-split-size=1M',
        '--summary-interval=5',
        '--seed-time=0',
        MAGNET_LINK
    ]
    # Run aria2c and stream its output to the notebook
    import subprocess
    process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
    try:
        for line in process.stdout:
            print(line, end='')
    except KeyboardInterrupt:
        # If you interrupt the cell (Ctrl+C) aria2 will be terminated
        process.terminate()
        raise
    ret = process.wait()
    print('aria2 exit code:', ret)




10/20 13:40:57 [NOTICE] Downloading 1 item(s)

10/20 13:40:58 [ERROR] Exception caught while loading DHT routing table from /root/.cache/aria2/dht.dat
Exception: [DHTRoutingTableDeserializer.cc:82] errorCode=1 Failed to load DHT routing table from /root/.cache/aria2/dht.dat

10/20 13:40:58 [NOTICE] IPv4 DHT: listening on UDP port 6968

10/20 13:40:58 [NOTICE] IPv4 BitTorrent: listening on TCP port 6892

10/20 13:40:58 [ERROR] IPv6 BitTorrent: failed to bind TCP port 6892
Exception: [SocketCore.cc:312] errorCode=1 Failed to bind a socket, cause: Name or service not known
[#90a097 0B/0B CN:0 SD:0 DL:0B]
[#90a097 0B/0B CN:0 SD:0 DL:0B]
[#90a097 0B/0B CN:0 SD:0 DL:0B]
[#90a097 0B/0B CN:0 SD:0 DL:0B]
 *** Download Progress Summary as of Mon Oct 20 13:41:03 2025 *** 
[#90a097 0B/0B CN:0 SD:0 DL:0B]
FILE: [MEMORY][METADATA]Miasma Chronicles [DODI Repack]
-------------------------------------------------------------------------------

[#90a097 0B/0B CN:0 SD:0 DL:0B]
[#90a097 0B/0B CN:0 SD:0 D

In [ ]:

# ---------- Cell 4: Alternate - Start aria2 in background (non-blocking) ----------
# This starts aria2 in the background so the notebook doesn't block. You
# can view logs or kill the process later.

from subprocess import Popen

MAGNET_LINK_BG = "MAGNET_LINK_HERE"

if MAGNET_LINK_BG != "MAGNET_LINK_HERE":
    logfile = os.path.join('/content', 'aria2_bg.log')
    # Build a shell command so aria2 will run detached via nohup
    shell_cmd = (
        'nohup aria2c '
        f"--dir={DRIVE_TORRENTS_PATH} "
        '--continue=true '
        '--max-connection-per-server=4 '
        '--split=4 '
        '--min-split-size=1M '
        '--summary-interval=5 '
        '--seed-time=0 '
        f"'{MAGNET_LINK_BG}' "
        f"> {logfile} 2>&1 & echo $!"
    )
    pid = os.popen(shell_cmd).read().strip()
    print('Started aria2 in background, PID =', pid)
    print('Logfile:', logfile)
else:
    print('Set MAGNET_LINK_BG to a real magnet link and re-run this cell to start in background.')



In [ ]:

# ---------- Cell 5: Utility - list files in torrents folder ----------

def list_torrents_folder(path=DRIVE_TORRENTS_PATH):
    for root, dirs, files in os.walk(path):
        level = root.replace(path, '').count(os.sep)
        indent = ' ' * 2 * level
        print(f"{indent}{os.path.basename(root)}/")
        for f in files:
            print(f"{indent}  {f}")

print('\nContents of the Drive torrents folder:')
list_torrents_folder()


In [ ]:


# ---------- Cell 6: Cleanup / stop background aria2 (if started) ----------
# If you started aria2 in background and want to stop it, find its PID and kill it.

import subprocess

def find_aria2_pids():
    out = subprocess.check_output(['pgrep', '-f', 'aria2c']).decode().strip()
    return out.splitlines() if out else []

pids = find_aria2_pids()
if pids:
    print('aria2 PIDs found:', pids)
    # To kill: uncomment the next line
    # subprocess.check_call(['kill', '-9'] + pids)
else:
    print('No aria2 processes found.')


# ---------- Notes ----------
# - Colab sessions may disconnect; long torrents are better handled on
#   a persistent machine (VPS, home server, or a dedicated seedbox).
# - This script writes the downloaded files directly into your Google Drive
#   mount. Drive sync may take some time if files are large.
# - Always obey copyright and only download content you are allowed to.
